In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Ingesta de la *Data*

In [ ]:
df = pd.read_csv('data/Transporte_Publico_Espana.csv')

#### Empezamos con la exploracion del *CSV* 

In [ ]:
df.head(3)

En un principio vemos que hay NaN y un valor de 999 minutos de retraso para el ID 1.
La fecha esta con año-mes-dia-hora-minutos-segundos

In [ ]:
df.info()

Hay que transformar la fecha a datetime.
Duracion_Viaje_Minutos no tiene el tipo correcto deberia ser numerico

In [ ]:
df.describe()

Retraso_Minutos presenta un valor maximo de 999 muy fuera de rango si tomamos max + std

In [ ]:
df.duplicated().sum() 

Hay valores duplicados !!

In [ ]:
df.isnull().sum()

In [ ]:
df_copia1 = df.copy()

#### Comenzamos a transformar !!

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['Duracion_Viaje_Minutos'] = pd.to_numeric(df['Duracion_Viaje_Minutos'], errors='coerce')

Dos opciones imputar valores o dropear los valores faltantes o Nan. (cantidad <1.5%)

In [ ]:
df = df.dropna(subset=['Numero_Pasajeros'])

In [ ]:
df['Duracion_Viaje_Minutos'] = df['Duracion_Viaje_Minutos'].fillna(df['Duracion_Viaje_Minutos'].mean())

Columna Retraso_Minutos: Eliminar 500 registros en un dataset de 100,000 representa solo el 0.5% del total, lo cual es un impacto mínimo en términos de representatividad.

In [ ]:
df = df[df['Retraso_Minutos'] != 999]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

Para poder buscar patrones por mes, dia y hora creo nuevas columnas a partir de la columna Fecha. Ademas 

In [ ]:
df['Mes'] = df['Fecha'].dt.month
df['Día'] = df['Fecha'].dt.day
df['Hora'] = df['Fecha'].dt.hour
df = df.drop(columns='Fecha')

In [ ]:
df.info()

In [ ]:
df.head(3)

In [ ]:
df_copia2 = df.copy

In [ ]:
df.shape

#### Analisis descriptivo de los retrasos

In [ ]:
df.Retraso_Minutos.describe()

La media de los retrasos es de aproximadamente 59 minutos, con una considerable variabilidad (std de 34.69 minutos), indicando que los retrasos pueden variar ampliamente.

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['Retraso_Minutos'], bins=30, color='blue', edgecolor='black')
plt.xlabel('Retraso (Minutos)')
plt.ylabel('Conteo')
plt.title('Distribución de Retrasos')
plt.show()


El histograma muestra una distribución uniforme de los retrasos en minutos, indicando que los retrasos están distribuidos de manera bastante equitativa a lo largo del rango de 0 a 120 minutos.

##### Segmentación por Variables (comparación)

In [ ]:
# Retraso Promedio por Ruta
average_delay_by_route = df.groupby('Ruta')['Retraso_Minutos'].mean().reset_index()
plt.figure(figsize=(12, 8))
plt.bar(average_delay_by_route['Ruta'], average_delay_by_route['Retraso_Minutos'], color='blue')
plt.xlabel('Ruta')
plt.ylabel('Retraso Promedio (Minutos)')
plt.title('Retraso Promedio por Ruta')
plt.xticks(rotation=40, ha='right') 
plt.grid(True) 
plt.show()


Dado que los retrasos son uniformes entre las rutas, podría ser útil investigar factores sistémicos, como la infraestructura general, la planificación de horarios o la gestión operativa

In [ ]:
# Retraso promedio por tipo de transporte
average_delay_by_transport = df.groupby('Tipo_Transporte')['Retraso_Minutos'].mean().reset_index()
plt.figure(figsize=(12, 8))
plt.bar(average_delay_by_transport['Tipo_Transporte'], average_delay_by_transport['Retraso_Minutos'], color='green')
plt.xlabel('Tipo de Transporte')
plt.ylabel('Retraso Promedio (Minutos)')
plt.title('Retraso Promedio por Tipo de Transporte')
plt.grid(True)
plt.show()


Los retrasos no parecen estar influenciados por el tipo de transporte, por lo que podría ser útil investigar factores externos o sistémicos que afecten a todos los modos de transporte de manera similar.

In [ ]:
# Retraso promedio por región
average_delay_by_region = df.groupby('Region')['Retraso_Minutos'].mean().reset_index()
plt.figure(figsize=(12, 8))
plt.bar(average_delay_by_region['Region'], average_delay_by_region['Retraso_Minutos'], color='red')
plt.xlabel('Región')
plt.ylabel('Retraso Promedio (Minutos)')
plt.title('Retraso Promedio por Región')
plt.xticks(rotation=90)
plt.grid(True)
plt.show()


Los retrasos no parecen estar influenciados por la ubicación geográfica, lo que apunta a la posibilidad de factores sistémicos o nacionales que afectan el transporte en general.

#### Análisis Temporal

In [ ]:
# Por Mes
plt.figure(figsize=(12, 6))
sns.boxplot(x='Mes', y='Retraso_Minutos', data=df)
plt.title('Distribución de Retrasos por Mes')
plt.xlabel('Mes')
plt.ylabel('Retraso en Minutos')
plt.show()


La distribución de los retrasos por mes es uniforme, indicando que no hay una variación significativa en los retrasos entre estos meses. Esto sugiere que los factores que causan los retrasos son consistentes a lo largo del tiempo en este periodo. Solo se trabajo con el mes de enero, Febrero y 11 dias del mes de Marzo.

In [ ]:
# Por día de la semana
average_delay_by_day = df.groupby('Dia_Semana')['Retraso_Minutos'].mean().reset_index()
plt.figure(figsize=(12, 8))
plt.bar(average_delay_by_day['Dia_Semana'], average_delay_by_day['Retraso_Minutos'], color='purple')
plt.xlabel('Día de la Semana')
plt.ylabel('Retraso Promedio (Minutos)')
plt.title('Retraso Promedio por Día de la Semana')
plt.grid(True)
plt.show()

 Los retrasos no parecen estar influenciados por el día de la semana, lo que apunta a la posibilidad de factores sistémicos que afectan el transporte de manera uniforme cada día.

In [ ]:
# Por Hora
plt.figure(figsize=(12, 6))
sns.lineplot(x='Hora', y='Retraso_Minutos', data=df, errorbar=None)
plt.title('Tendencia de Retrasos por Hora del Día')
plt.xlabel('Hora del Día')
plt.ylabel('Retraso en Minutos')
plt.xticks(ticks=range(0, 24, 2))
plt.show()


La variabilidad en los retrasos a lo largo del día sugiere que hay factores temporales que influyen en la puntualidad del transporte, posiblemente relacionados con la demanda de pasajeros, la congestión del tráfico o la programación de los servicios.
Considerar ajustes en los horarios de los servicios de transporte durante las horas pico identificadas para mitigar los retrasos.

In [ ]:
df.Ruta.unique()

In [ ]:
df.head(3)

#### Conectamos a MySQL

In [ ]:
#! pip install mysql-connector-python

In [ ]:
import mysql.connector
import os

db_config_no_db = {
    'user': '*****',
    'password': '******', 
    'host': 'localhost'
}
conn = mysql.connector.connect(**db_config_no_db)
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS transporte_publico")
conn.commit()
conn.close()

#### Creamos las tablas en Python y las generamos en MySQL

In [ ]:
db_config = {
    'user': '****',
    'password': '*****',
    'host': 'localhost'
}
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS transporte_publico")
cursor.execute("USE transporte_publico")

cursor.execute('''
CREATE TABLE IF NOT EXISTS rutas (
    ID_Ruta INT AUTO_INCREMENT PRIMARY KEY,
    Ruta VARCHAR(255) UNIQUE
)
''')
cursor.execute('''
CREATE TABLE IF NOT EXISTS tipos_transporte (
    ID_Tipo_Transporte INT AUTO_INCREMENT PRIMARY KEY,
    Tipo_Transporte VARCHAR(50) UNIQUE
)
''')
cursor.execute('''
CREATE TABLE IF NOT EXISTS regiones (
    ID_Region INT AUTO_INCREMENT PRIMARY KEY,
    Region VARCHAR(50) UNIQUE
)
''')
cursor.execute('''
CREATE TABLE IF NOT EXISTS viajes (
    ID_Viaje INT PRIMARY KEY,
    ID_Ruta INT,
    Numero_Pasajeros FLOAT,
    Duracion_Viaje_Minutos FLOAT,
    Retraso_Minutos INT,
    ID_Tipo_Transporte INT,
    ID_Region INT,
    Dia_Semana VARCHAR(20),
    Mes INT,
    Día INT,
    Hora INT,
    FOREIGN KEY (ID_Ruta) REFERENCES rutas(ID_Ruta),
    FOREIGN KEY (ID_Tipo_Transporte) REFERENCES tipos_transporte(ID_Tipo_Transporte),
    FOREIGN KEY (ID_Region) REFERENCES regiones(ID_Region)
)
''')
conn.commit()


#### Llenamos las tablas desde python en MySQL 

In [ ]:
for ruta in df['Ruta'].unique():
    cursor.execute('INSERT IGNORE INTO rutas (Ruta) VALUES (%s)', (ruta,))
for tipo in df['Tipo_Transporte'].unique():
    cursor.execute('INSERT IGNORE INTO tipos_transporte (Tipo_Transporte) VALUES (%s)', (tipo,))
for region in df['Region'].unique():
    cursor.execute('INSERT IGNORE INTO regiones (Region) VALUES (%s)', (region,))
conn.commit()


In [ ]:
datos_viajes = df.to_records(index=False).tolist()
consulta_insercion_viajes = '''
INSERT INTO viajes (
    ID_Viaje, ID_Ruta, Numero_Pasajeros, Duracion_Viaje_Minutos, Retraso_Minutos, 
    ID_Tipo_Transporte, ID_Region, Dia_Semana, Mes, Día, Hora
) VALUES (
    %s, 
    (SELECT ID_Ruta FROM rutas WHERE Ruta = %s),
    %s, %s, %s, 
    (SELECT ID_Tipo_Transporte FROM tipos_transporte WHERE Tipo_Transporte = %s),
    (SELECT ID_Region FROM regiones WHERE Region = %s),
    %s, %s, %s, %s
)
'''
datos_viajes_preparados = [
    (
        row.ID_Viaje, row.Ruta, row.Numero_Pasajeros, row.Duracion_Viaje_Minutos, row.Retraso_Minutos, 
        row.Tipo_Transporte, row.Region, row.Dia_Semana, row.Mes, row.Día, row.Hora
    )
    for row in df.itertuples()
]

cursor.executemany(consulta_insercion_viajes, datos_viajes_preparados)
conn.commit()
conn.close()


#### Guardamos los datos en un csv

In [ ]:
df.to_csv('transporte_publico_limpio.csv', index=False)

#### Dado que no hay patrones visibles, verificamos por correlacion
Verificacion de la correlacion entre las columnas

Tranformo todos los datos a numericos 

In [ ]:
def transform_column_to_int(df, column_name):
    categories = df[column_name].unique()
    category_map = {category: idx for idx, category in enumerate(categories)}
    df[column_name] = df[column_name].apply(lambda x: category_map[x])
    df[column_name] = df[column_name].astype('int64')
    return df


columns_to_transform = ['Ruta', 'Tipo_Transporte', 'Region', 'Dia_Semana']
for column in columns_to_transform:
    df = transform_column_to_int(df, column)

In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Matriz de Correlación')
plt.show()

La mayoría de las correlaciones entre variables son muy bajas, cercanas a cero, lo que indica que no hay una relación lineal fuerte entre estas variables.